In [1]:
"""Dependencies"""
# !pip install import-ipynb

#access drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial

x_path= "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/ohc3.csv"
y_path= "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/depVar"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


In [2]:
%run Splitting_data.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial # Change path to yours'
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


In [3]:
X_train, X_val, y_train, y_val = training_set(x_path, y_path)

In [ ]:
for column in X_train.columns:
  print(X_train[column].dtype)

In [9]:
y_train.dtype

dtype('int64')

In [ ]:
for column in X_train.columns:
  if X_train[column].dtype == "object":
    print("yes")
  else:
    print("NO")
#no column that is of type "object"

In [4]:
y_train = y_train.fillna("")
X_train = X_train.fillna("")
y_val = y_val.fillna("")
X_val = X_val.fillna("")

In [8]:
from sklearn import preprocessing

le=preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)

In [10]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_val, y_val)


ValueError: ignored

In [ ]:
clf.predict(X_val)
clf.predict_proba(X[:2, :])
clf.score(X, y)

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp.fit_transform(X_train)
X_val = imp.fit_transform(X_val)